In [548]:
from typing import Tuple, List, Dict
from itertools import chain
import json
from collections import ChainMap
import requests
from copy import deepcopy
from itertools import permutations
import csv

In [627]:
stage_one = [5244, 5245, 5246]

meta = [
    {'editors': ('Колмаков', 'Саксонов'), 'id': 5244, 'group': 'A'},
    {'editors': ('Колмаков', 'Саксонов'), 'id': 5245, 'group': 'B'},
    {'editors': ('Колмаков', 'Саксонов'), 'id': 5246, 'group': 'C'},
    {'editors': ('Иванов', 'Кудрявцев'), 'id': 5252, 'group': 'A'},
    {'editors': ('Иванов', 'Кудрявцев'), 'id': 5253, 'group': 'B'},
    {'editors': ('Иванов', 'Кудрявцев'), 'id': 5254, 'group': 'C'},
    {'editors': ('Абрамов', 'Терентьев'), 'id': 5255, 'group': 'A'},
    {'editors': ('Абрамов', 'Терентьев'), 'id': 5256, 'group': 'B'},
    {'editors': ('Абрамов', 'Терентьев'), 'id': 5257, 'group': 'C'},
    {'editors': ('Скиренко', 'Карпук'), 'id': 5259, 'group': 'A'},
    {'editors': ('Скиренко', 'Карпук'), 'id': 5260, 'group': 'B'},
    {'editors': ('Скиренко', 'Карпук'), 'id': 5261, 'group': 'C'},
]

actual_editors_sequence = ', '.join(' и '.join(e for e in pair) 
                          for pair in [sorted(meta[i]['editors']) for i in (0, 3, 6, 9)])
actual_editors_sequence

'Колмаков и Саксонов, Иванов и Кудрявцев, Абрамов и Терентьев, Карпук и Скиренко'

In [539]:
def parse_mask(mask: str) -> Tuple[int, int]:
    first_tour, second_tour = mask[:15], mask[15:]
    return first_tour.count('1'), second_tour.count('1')

In [162]:
def load_results(meta_dict: Dict) -> List:
    url = f'https://rating.chgk.info/api/tournaments/{meta_dict["id"]}/list.json'
    resp = requests.get(url).json()
    for team in resp:
        team['meta'] = meta_dict
    return resp

In [163]:
results = [load_results(dict_) for dict_ in meta]

In [164]:
flat_results = list(chain.from_iterable(results))
team_ids = set(result['idteam'] for result in flat_results)
team_dicts = [[dict_ for dict_ in flat_results if dict_['idteam'] == team_id] 
              for team_id in team_ids]

In [491]:
class Team:
    def __init__(self, dicts: List[Dict]):
        self.name = dicts[0]['base_name']
        self.id = dicts[0]['idteam']
        self.results = self.parse_results(dicts)
        self.starting_group = dicts[0]['meta']['group']
        self.rating = int(dicts[0]['tech_rating_rg'])
        self.sum = 0
        self.sums = []
        self.place = None
        
    def __repr__(self):
        return f"{self.place}. {self.name} {self.current_sum}"
        return json.dumps(self.__dict__, ensure_ascii=False)
    
    @staticmethod
    def parse_results(stages: List[Dict]):
        zipped = [dict(zip(stage['meta']['editors'], parse_mask(stage['mask'])))
                   for stage in stages]
        return {**zipped[0], **zipped[1], **zipped[2], **zipped[3]}
    
    @property
    def previous_sum(self):
        return sum(self.sums[:-1])
    
    @property
    def current_sum(self):
        return self.sums[-1] if self.sums else 0
            

In [492]:
teams = [Team(dicts) for dicts in team_dicts]

In [565]:
group_a = [t for t in teams if t.starting_group == 'A']
group_b = [t for t in teams if t.starting_group == 'B']
group_c = [t for t in teams if t.starting_group == 'C']
move_a_b = round((len(group_a) + 0.1) / 4)
move_b_c = round((len(group_b) + 0.1) / 4)

In [497]:
def generate_tournaments(editors):
    perms = permutations(editors)
    paired_perms = ((p[0:2], p[2:4], p[4:6], p[6:8]) for p in perms)
    sorted_pairs = (tuple(tuple(sorted(pair)) for pair in perm) for perm in paired_perms)
    return set(sorted_pairs)

In [540]:
editors = sorted(chain.from_iterable(set(tour['editors'] for tour in meta)))
tournaments = list(generate_tournaments(editors))

In [647]:
def run(tournament, a, b, c):
    for stage in range(3):
        run_stage(tournament[stage], a, b, c)
        a, b, c = move_teams(*sort_preliminary_stage(a, b, c))

    run_stage(tournament[3], a, b, c)
    all_teams = place_teams(a, b, c)
    editors = ', '.join(' и '.join(e for e in pair) for pair in tournament)
    final_editors = ' и '.join(tournament[3])
    return [(team.name, 
             team.place, 
             editors,
#              team.sums, 
             team.current_sum, 
             final_editors,
             (editors == actual_editors_sequence)) 
            for team in all_teams]
        
        
def run_stage(editors_pair, a, b, c):
    for team in [*a, *b, *c]:
        team.sums.append(team.results[editors_pair[0]] + team.results[editors_pair[1]])
    
def sorting_key(team: Team):
    return (team.current_sum, team.previous_sum, -team.rating)

def sort_preliminary_stage(a, b, c):
    return (sorted(a, key=sorting_key, reverse=True),
            sorted(b, key=sorting_key, reverse=True),
            sorted(c, key=sorting_key, reverse=True))


def place_teams(a, b, c):
    sorted_final = sort_final(a)
    
    sorted_non_final = (*sorted(b, key=sorting_key, reverse=True),
            *sorted(c, key=sorting_key, reverse=True))
    
    placed = zip(range(len(sorted_final), len(a) + len(b) + len(c) + 1), sorted_non_final)
    for place, team in placed:
        team.place = place

    return (*sorted_final, *sorted_non_final)

def sort_final(teams):
    first, second, third = sorted((t.current_sum for t in teams), reverse=True)[:3]
                 
    for t in teams:
        if t.current_sum == first:
            t.place = 1
        elif t.current_sum == second and second < first:
            t.place = 2
        elif t.current_sum == third and third < second:
            t.place = 3
                 
    not_placed = sorted((team for team in teams if team.place is None), 
                        key=sorting_key, reverse=True)
    for index, team in enumerate(not_placed, len(teams) - len(not_placed)):
        team.place = index
                 
    return sorted(teams, key=lambda t: t.place)


def move_teams(a, b, c):
    new_a = [*a[:len(a) - move_a_b], *b[:move_a_b]]
    new_b = [*a[len(a) - move_a_b:], *b[move_a_b:len(b) - move_b_c], *c[:move_b_c]]
    new_c = [*b[len(b) - move_b_c:], *c[move_b_c:]]
    return new_a, new_b, new_c

In [648]:
results = list(chain.from_iterable(run(t, deepcopy(group_a), deepcopy(group_b), deepcopy(group_c)) 
                                   for t in tournaments))

('Сборная Кирибати',
 1,
 'Кудрявцев и Терентьев, Колмаков и Саксонов, Абрамов и Иванов, Карпук и Скиренко',
 [19, 24, 21, 21],
 21,
 'Карпук и Скиренко',
 False)

In [649]:
with open('2019.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(("Команда", "Место", "Редакторы", 
                     --"Суммы по турам", 
                     "Сумма в финале",
                     "Настоящий ЧМ"))
    writer.writerows(results)

In [569]:
from collections import Counter

In [584]:
str_res = [t for t in results if t[0] == 'Страпелька']
len(str_res)
str_places = [t[1] for t in str_res]

In [585]:
c = Counter(res[1] for res in str_res)
c.most_common()

[(13, 531),
 (12, 252),
 (11, 250),
 (9, 210),
 (14, 200),
 (10, 197),
 (15, 143),
 (8, 128),
 (16, 113),
 (17, 91),
 (3, 70),
 (18, 68),
 (7, 68),
 (5, 42),
 (6, 33),
 (21, 26),
 (22, 25),
 (23, 18),
 (2, 14),
 (19, 13),
 (25, 11),
 (20, 5),
 (24, 4),
 (1, 4),
 (27, 2),
 (28, 1),
 (26, 1)]

In [590]:
from statistics import median
from collections import defaultdict

In [586]:
median(str_places)

12.0

In [589]:
champions = [t[0] for t in results if t[1] == 1]
c = Counter(champions)
c.most_common()

[('Сборная Кирибати', 1405),
 ('Борский корабел', 995),
 ('Братья', 206),
 ('Зигзаг', 126),
 ('Эрудиты', 103),
 ('Команда Губанова', 90),
 ('Рабочее название', 89),
 ('Тачанка', 88),
 ('Страпелька', 4)]

In [640]:
places = defaultdict(list)
for t in results:
    places[t[0]].append(t[1])
    
medians = [(k, int(median(v))) for k, v in places.items()]
actual = {t[0]: t[1] for t in results if t[5] is True}
sorted(((k, v, actual[k]) for k, v in medians), key=lambda x: x[2])

[('Сборная Кирибати', 1, 1),
 ('Команда Губанова', 4, 2),
 ('Рабочее название', 4, 2),
 ('Эрудиты', 6, 4),
 ('Ксеп', 6, 5),
 ('Борский корабел', 2, 6),
 ('Зигзаг', 8, 7),
 ('Братья', 8, 8),
 ('Мираж', 10, 9),
 ('Арагаст', 12, 10),
 ('Напряжены у всех на глазах', 14, 11),
 ('Вжух', 12, 12),
 ('Страпелька', 12, 13),
 ('X-promt', 17, 13),
 ('Зненацька', 18, 14),
 ('Keisecker', 13, 15),
 ('Гвардия', 21, 16),
 ('Хронически разумные United', 25, 17),
 ('Пахом Пихай', 16, 18),
 ('Тачанка', 10, 19),
 ('Афина', 20, 20),
 ('Как-то так', 31, 21),
 ('Сборная Нидерландов', 30, 22),
 ('Ворона и Медведы', 41, 23),
 ('Приматы', 19, 24),
 ('Номер 6', 23, 25),
 ('Церебрум', 23, 26),
 ('Кьяроскуро', 25, 27),
 ('Пентиум', 27, 28),
 ('Язык Чау-Чау', 32, 29),
 ('Бедлам', 27, 30),
 ('Панические атаки', 30, 31),
 ('Веретено', 42, 32),
 ('ИЖ Юпитер 5', 42, 33),
 ('Gambler', 35, 34),
 ('DimTeam', 44, 35),
 ('Giper-Turbo', 51, 36),
 ('Норманны', 34, 37),
 ('Мистерия', 46, 38),
 ('Тёмный лес', 33, 39),
 ('Eclipse

In [646]:
[r for r in results if r[-1] is True]

[('Сборная Кирибати',
  1,
  'Колмаков и Саксонов, Иванов и Кудрявцев, Абрамов и Терентьев, Карпук и Скиренко',
  [24, 22, 18, 21],
  21,
  True),
 ('Рабочее название',
  2,
  'Колмаков и Саксонов, Иванов и Кудрявцев, Абрамов и Терентьев, Карпук и Скиренко',
  [17, 19, 18, 20],
  20,
  True),
 ('Команда Губанова',
  2,
  'Колмаков и Саксонов, Иванов и Кудрявцев, Абрамов и Терентьев, Карпук и Скиренко',
  [19, 16, 21, 20],
  20,
  True),
 ('Эрудиты',
  3,
  'Колмаков и Саксонов, Иванов и Кудрявцев, Абрамов и Терентьев, Карпук и Скиренко',
  [19, 16, 19, 19],
  19,
  True),
 ('Ксеп',
  4,
  'Колмаков и Саксонов, Иванов и Кудрявцев, Абрамов и Терентьев, Карпук и Скиренко',
  [18, 18, 19, 18],
  18,
  True),
 ('Борский корабел',
  5,
  'Колмаков и Саксонов, Иванов и Кудрявцев, Абрамов и Терентьев, Карпук и Скиренко',
  [24, 21, 22, 17],
  17,
  True),
 ('Зигзаг',
  6,
  'Колмаков и Саксонов, Иванов и Кудрявцев, Абрамов и Терентьев, Карпук и Скиренко',
  [17, 19, 18, 17],
  17,
  True),
 ('